In [1]:
!pip3 install torch torchaudio torchvision torchtext torchdata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 18.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim

# Name initialization

In [4]:
first_names = ["Ali", "Zahra", "Reza", "Sara", "Mohammad", "Fatemeh", "Hossein", "Maryam", "Mehdi", "Narges", "Hamed", "Roya"]
last_names = ["Ahmadi", "Hosseini", "Karimi", "Rahimi", "Hashemi", "Ebrahimi", "Moradi", "Mohammadi", "Rostami", "Fazeli", "Hosseinzadeh", "Niknam"]
# Set seed for reproducibility
torch.manual_seed(10)
np.random.seed(10)
random.seed(10)
num_participants=24

def generate_random_names(first_names, last_names, num_participants):
    random_names = []
    name_dict = dict()
    while len(random_names) < num_participants:
        first_name = random.choice(first_names)
        last_name = random.choice(last_names)
        if not f"{first_name} {last_name}" in random_names:
            random_names.append(f"{first_name}_{last_name}")
    for idx,i in enumerate(random_names):
      name_dict.update({random_names[idx]:idx})
    return random_names, name_dict

random_names, name_dict = generate_random_names(first_names, last_names, 24)



def input_init(random_names, name_dict, hostility_occ):
  # Create an empty 24x24 hostility matrix
  hostility_mat = np.zeros((len(random_names),len(random_names)))
  # Randomly assign 40 hostile relationships
  hostility_pairs = random.sample([(i, j) for i in range(num_participants) for j in range(num_participants) if i != j], hostility_occ)
  for i, j in hostility_pairs:
      hostility_mat[i, j] = 1
  # Make hostility symmetric by averaging with the transpose
  hostility_mat = (hostility_mat + hostility_mat.T) / 2

  # Convert the hostility matrix to a PyTorch tensor
  hostility_mat = torch.tensor(hostility_mat, dtype=torch.float32)
  return hostility_mat
hostility_mat= input_init(random_names, name_dict, 40)
print(hostility_mat)

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.5000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5000,
         0.0000, 0.0000, 0.0000, 0.5000, 0.5000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000

In [17]:
class SeatingNetwork(nn.Module):
    def __init__(self):
        super(SeatingNetwork, self).__init__()
        self.fc1 = nn.Linear(24, 512)  # Adjust input to match 24 participants
        self.fc2 = nn.Linear(512,)

    def forward(self, x):
        # Forward pass through the network
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.softmax(self.fc4(x), dim=1)
        return x

In [13]:
class CustomLoss(nn.Module):
    def forward(self, output, H):
        """
        Custom loss function to minimize the number of hostile pairs seated next to each other.
        """
        # Convert output probabilities to seating arrangement
        seating = torch.argsort(output, dim=1)

        # Calculate loss
        loss = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)
        for i in range(num_participants):
            for j in range(i + 1, num_participants):
                if H[seating[0][i], seating[0][j]] == 1 and abs(i - j) == 1:
                    loss = loss + 1  # Accumulate loss

        return loss

def custom_loss_function(output, H):
    """
    Custom loss function to minimize the number of hostile pairs seated next to each other.
    """
    # Convert output probabilities to seating arrangement
    seating = torch.argsort(output, dim=1)

    # Calculate loss
    loss = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)
    for i in range(num_participants):
        for j in range(i + 1, num_participants):
            if H[seating[0][i], seating[0][j]] == 1 and abs(i - j) == 1:
                loss = loss + 1  # Accumulate loss

    return loss

In [9]:
# Generate unique random initial seating arrangements
X_train = np.array([np.random.permutation(num_participants) for _ in range(1000)])  # 1000 unique samples
X_train = np.unique(X_train, axis=0)  # Ensure all samples are unique

# Ensure there are enough unique samples
if len(X_train) < 1000:
    raise ValueError("Not enough unique samples generated. Increase the number of attempts or reduce sample size.")

# Convert to PyTorch tensor
X_train = torch.tensor(X_train, dtype=torch.float32)

# Test to ensure uniqueness
assert X_train.shape[0] == len(set([tuple(row) for row in X_train.numpy()])), "Duplicate samples found in training data"

# Convert X_train to one-hot encoding to match the model input expectations
X_train_one_hot = torch.zeros(X_train.shape[0], num_participants, num_participants)
for i, arrangement in enumerate(X_train):
    X_train_one_hot[i, torch.arange(num_participants), arrangement.long()] = 1

In [38]:
for idx,_ in enumerate(X_train):
  if 1<idx<5:
    print(X_train[idx])

tensor([ 0.,  4., 17., 23.,  5., 12.,  2., 15.,  9., 20., 18.,  7., 19., 14.,
         3.,  6., 21.,  8., 13., 10.,  1., 22., 16., 11.])
tensor([ 0.,  6.,  1., 21.,  3.,  9., 14., 23., 17.,  4., 10., 19.,  2.,  5.,
        22., 15., 13., 16.,  8., 11.,  7., 20., 12., 18.])
tensor([ 0.,  6., 12., 16.,  5., 22., 11., 19., 18.,  2., 17., 20.,  9.,  1.,
         8.,  3., 23., 13., 14., 15., 21., 10.,  4.,  7.])


In [18]:
# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = CustomLoss()

# Function to visualize the seating arrangement
def visualize_seating(seating):
    fig, ax = plt.subplots()
    seating_matrix = np.zeros((4, 6))
    for idx, seat in enumerate(seating):
        row = idx // 6
        col = idx % 6
        seating_matrix[row, col] = seat + 1  # Add 1 for better visualization (1-24 instead of 0-23)

    ax.matshow(seating_matrix, cmap='viridis')
    for (i, j), val in np.ndenumerate(seating_matrix):
        ax.text(j, i, int(val), ha='center', va='center', color='white')

    plt.show()

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()

    total_loss = 0
    for i in range(len(X_train)):
        optimizer.zero_grad()

        # Forward pass
        output = model(X_train[i].unsqueeze(0))

        # Compute loss
        loss = loss_fn(output, hostility_mat)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(X_train)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss}")

    # Check if loss is zero and stop training
    if average_loss == 0:
        break

# Final evaluation and visualization
model.eval()
with torch.no_grad():
    output = model(X_train[0].unsqueeze(0))
    final_loss = custom_loss_function(output, hostility_mat)
    final_seating = torch.argsort(output, dim=1).numpy()[0]

    print("Final Seating Arrangement:", final_seating)
    print("Final Loss (Number of Hostile Pairs):", final_loss.item())

    # Visualize the final seating arrangement
    visualize_seating(final_seating)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x24 and 576x512)

##